In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import sklearn
from skimage.transform import resize

In [2]:
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
def resize_img(data):
    output = np.zeros((data.shape[0],64))
    for i, img in enumerate(data):
        output[i]=resize(img, (8,8)).flatten()
    return output

x_train_small = resize_img(x_train)
y_train_small = resize_img(y_train)
x_test_small = resize_img(x_test)
y_test_small = resize_img(y_test)

In [4]:
# NORMALIZE!
x_train = x_train_small.astype("float32") * 16.0
x_train = np.reshape(x_train, (-1, 64))

x_test = x_test_small.astype("float32") * 16.0
x_test = np.reshape(x_test, (-1, 64))

In [5]:
np.max(x_test)

15.87451

In [6]:
from sklearn.decomposition import PCA

In [7]:
from tensorflow.python.keras.models import load_model
import onnx
import keras2onnx

In [10]:
teacher_model = load_model('mnist8x8_100_80_60_40_20_10_eps1.h5')
student_model = load_model('mnist8x8_100_80_60_40_20_10_eps1.h5')

In [11]:
import pickle
properties = pickle.load(open("to_verify.pickle","rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'to_verify.pickle'

In [14]:
import copy
from keras.layers import Concatenate,Dense

def build_small_input_net(dim, x_train, x_test, y_test, teacher, student):
    pca = sklearn.decomposition.PCA(n_components=dim,whiten=True)
    pca.fit(x_train)
    print(f"Explained Variance: {np.sum(pca.explained_variance_ratio_)}")
    
    def weight_mat_init(shape, dtype=None):
        assert len(shape)==2 and (shape[0],shape[1])==pca.components_.shape, f"{shape},{pca.components_.shape}"
        return pca.components_
    
    layers = [
            keras.Input(shape=(dim,)),
            Dense(64, activation='linear', use_bias=False, trainable=False, kernel_initializer=weight_mat_init, name="pca")
        ]
    x_test_small = pca.transform(x_test)
    print("Teacher...")
    teacher_layers = copy.copy(layers)
    for l in teacher.layers:
        teacher_layers.append(l)
    pca_teacher_model = keras.Sequential(
        teacher_layers,
        name="pca_teacher",
    )
    pca_teacher_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()])
    pca_teacher_model.summary()
    pca_teacher_model.evaluate(x_test_small, y_test)
    print("Student...")
    student_layers = copy.copy(layers)
    for l in student.layers:
        student_layers.append(l)
    pca_student_model = keras.Sequential(student_layers, name="pca_student")
    pca_student_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()])
    pca_student_model.summary()
    pca_student_model.evaluate(x_test_small, y_test)
    return pca_teacher_model, pca_student_model, pca

In [16]:
res=build_small_input_net(50, x_train_small, x_test_small, y_test, teacher_model, student_model)
    

Explained Variance: 0.9995889287025757
Teacher...
Model: "pca_teacher"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pca (Dense)                  (None, 64)                3200      
_________________________________________________________________
dense (Dense)                (None, 100)               6500      
_________________________________________________________________
dense_1 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_2 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                2440      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                820       
_________________________________________________________________
dense

In [18]:
teacher, student, pca = res

In [19]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.
        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                teacher_predictions,
                student_predictions,
            )
            loss = distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [20]:
teacher.fit(pca.transform(x_train), y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3799 - sparse_categorical_accuracy: 0.9370
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1160 - sparse_categorical_accuracy: 0.9704
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0997 - sparse_categorical_accuracy: 0.9744
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0952 - sparse_categorical_accuracy: 0.9772
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0990 - sparse_categorical_accuracy: 0.9774
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1048 - sparse_categorical_accuracy: 0.9764
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1076 - sparse_categorical_accuracy: 0.9766
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1182 - sparse_categorical_accuracy: 0.9754
Epoch 9/10
1875/1875 [==========

In [22]:
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.MeanSquaredError(),
    alpha=0.1,
    temperature=10,
)
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
callback = keras.callbacks.LearningRateScheduler(scheduler)
# Distill teacher to student
distiller.fit(pca.transform(x_train), y_train, epochs=10, callbacks=[callback])

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.8566 - student_loss: 2.1735 - distillation_loss: 1935.2842 - lr: 0.0010
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9082 - student_loss: 0.7776 - distillation_loss: 871.3487 - lr: 0.0010
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9242 - student_loss: 0.6258 - distillation_loss: 703.6566 - lr: 0.0010
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9293 - student_loss: 0.5722 - distillation_loss: 608.6416 - lr: 0.0010
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9343 - student_loss: 0.5211 - distillation_loss: 548.1563 - lr: 0.0010
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9364 - student_loss: 0.4989 - distilla

In [23]:
distiller.fit(pca.transform(x_train), y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9459 - student_loss: 0.3940 - distillation_loss: 290.8484
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9470 - student_loss: 0.3793 - distillation_loss: 277.8338
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9474 - student_loss: 0.3762 - distillation_loss: 274.3162
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9485 - student_loss: 0.3648 - distillation_loss: 265.0067
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9479 - student_loss: 0.3672 - distillation_loss: 257.3004
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - sparse_categorical_accuracy: 0.9493 - student_loss: 0.3503 - distillation_loss: 250.9722
Epoch 7/10
1875/1875 [========================

In [24]:
distiller.evaluate(pca.transform(x_test), y_test)

313/313 [==============================] - 0s 1ms/step - sparse_categorical_accuracy: 0.9461 - student_loss: 0.5268


0.9460999965667725

In [25]:
teacher.evaluate(pca.transform(x_test), y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3771 - sparse_categorical_accuracy: 0.9629


[0.37712711095809937, 0.9628999829292297]

In [33]:
teacher.save("mnist8x8_100_80_60_40_20_10_eps1_pca.h5")

In [34]:
student.save("mnist8x8_100_80_60_40_20_10_eps1_pca-mirror.h5")

In [35]:
from tensorflow.python.keras.models import load_model
import onnx
import keras2onnx

In [36]:
model = teacher#load_model('mnist8x8_100_80_60_40_20_10_eps1_pca.h5')
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, "mnist8x8_100_80_60_40_20_10_eps1_pca.onnx")

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 25 -> 19
The maximum opset needed by this model is only 9.


In [37]:
model = student#load_model('mnist8x8_100_80_60_40_20_10_eps1_pca-mirror.h5')
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, "mnist8x8_100_80_60_40_20_10_eps1_pca-mirror.onnx")

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 25 -> 19
The maximum opset needed by this model is only 9.


In [49]:
PROPERTY= {}
PROPERTY["8000.1"]=[pca.transform(np.array([[0.1, 0.1, 3.1, 13.1, 13.1, 3.1, 0.1, 0.1, 0.1, 1.1, 13.1, 11.1, 9.1, 12.1, 1.1, 0.1, 0.1, 5.1, 14.1, 2.1, 0.1, 10.1, 4.1, 0.1, 0.1, 6.1, 11.1, 0.1, 0.1, 6.1, 7.1, 0.1, 0.1, 6.1, 10.1, 0.1, 0.1, 5.1, 8.1, 0.1, 0.1, 3.1, 13.1, 1.1, 0.1, 8.1, 7.1, 0.1, 0.1, 0.1, 12.1, 8.1, 7.1, 14.1, 3.1, 0.1, 0.1, 0.1, 4.1, 13.1, 14.1, 7.1, 0.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 2.9, 12.9, 12.9, 2.9, 0.0, 0.0, 0.0, 0.9, 12.9, 10.9, 8.9, 11.9, 0.9, 0.0, 0.0, 4.9, 13.9, 1.9, 0.0, 9.9, 3.9, 0.0, 0.0, 5.9, 10.9, 0.0, 0.0, 5.9, 6.9, 0.0, 0.0, 5.9, 9.9, 0.0, 0.0, 4.9, 7.9, 0.0, 0.0, 2.9, 12.9, 0.9, 0.0, 7.9, 6.9, 0.0, 0.0, 0.0, 11.9, 7.9, 6.9, 13.9, 2.9, 0.0, 0.0, 0.0, 3.9, 12.9, 13.9, 6.9, 0.0, 0.0]]))]
PROPERTY["8100.1"]=[pca.transform(np.array([[0.1, 0.1, 0.1, 10.1, 9.1, 0.1, 0.1, 0.1, 0.1, 0.1, 5.1, 15.1, 6.1, 0.1, 0.1, 0.1, 0.1, 0.1, 11.1, 11.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1.1, 13.1, 8.1, 3.1, 1.1, 0.1, 0.1, 0.1, 1.1, 15.1, 14.1, 14.1, 12.1, 3.1, 0.1, 0.1, 0.1, 14.1, 10.1, 3.1, 6.1, 12.1, 0.1, 0.1, 0.1, 8.1, 13.1, 3.1, 7.1, 14.1, 1.1, 0.1, 0.1, 0.1, 8.1, 14.1, 14.1, 8.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 0.0, 9.9, 8.9, 0.0, 0.0, 0.0, 0.0, 0.0, 4.9, 14.9, 5.9, 0.0, 0.0, 0.0, 0.0, 0.0, 10.9, 10.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 12.9, 7.9, 2.9, 0.9, 0.0, 0.0, 0.0, 0.9, 14.9, 13.9, 13.9, 11.9, 2.9, 0.0, 0.0, 0.0, 13.9, 9.9, 2.9, 5.9, 11.9, 0.0, 0.0, 0.0, 7.9, 12.9, 2.9, 6.9, 13.9, 0.9, 0.0, 0.0, 0.0, 7.9, 13.9, 13.9, 7.9, 0.0]]))]
PROPERTY["8200.1"]=[pca.transform(np.array([[0.1, 0.1, 8.1, 14.1, 15.1, 14.1, 7.1, 0.1, 0.1, 0.1, 7.1, 8.1, 9.1, 15.1, 8.1, 0.1, 0.1, 0.1, 0.1, 0.1, 5.1, 14.1, 3.1, 0.1, 0.1, 0.1, 5.1, 9.1, 14.1, 13.1, 4.1, 0.1, 0.1, 1.1, 10.1, 14.1, 15.1, 12.1, 4.1, 0.1, 0.1, 0.1, 2.1, 13.1, 8.1, 2.1, 0.1, 0.1, 0.1, 0.1, 6.1, 15.1, 2.1, 0.1, 0.1, 0.1, 0.1, 0.1, 10.1, 10.1, 0.1, 0.1, 0.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 7.9, 13.9, 14.9, 13.9, 6.9, 0.0, 0.0, 0.0, 6.9, 7.9, 8.9, 14.9, 7.9, 0.0, 0.0, 0.0, 0.0, 0.0, 4.9, 13.9, 2.9, 0.0, 0.0, 0.0, 4.9, 8.9, 13.9, 12.9, 3.9, 0.0, 0.0, 0.9, 9.9, 13.9, 14.9, 11.9, 3.9, 0.0, 0.0, 0.0, 1.9, 12.9, 7.9, 1.9, 0.0, 0.0, 0.0, 0.0, 5.9, 14.9, 1.9, 0.0, 0.0, 0.0, 0.0, 0.0, 9.9, 9.9, 0.0, 0.0, 0.0, 0.0]]))]
PROPERTY["8300.1"]=[pca.transform(np.array([[0.1, 0.1, 5.1, 14.1, 8.1, 0.1, 0.1, 0.1, 0.1, 1.1, 14.1, 12.1, 13.1, 8.1, 0.1, 0.1, 0.1, 4.1, 14.1, 2.1, 4.1, 14.1, 2.1, 0.1, 0.1, 5.1, 12.1, 0.1, 0.1, 9.1, 6.1, 0.1, 0.1, 6.1, 11.1, 0.1, 0.1, 8.1, 7.1, 0.1, 0.1, 4.1, 13.1, 0.1, 0.1, 10.1, 7.1, 0.1, 0.1, 0.1, 14.1, 8.1, 8.1, 14.1, 3.1, 0.1, 0.1, 0.1, 5.1, 14.1, 14.1, 6.1, 0.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 4.9, 13.9, 7.9, 0.0, 0.0, 0.0, 0.0, 0.9, 13.9, 11.9, 12.9, 7.9, 0.0, 0.0, 0.0, 3.9, 13.9, 1.9, 3.9, 13.9, 1.9, 0.0, 0.0, 4.9, 11.9, 0.0, 0.0, 8.9, 5.9, 0.0, 0.0, 5.9, 10.9, 0.0, 0.0, 7.9, 6.9, 0.0, 0.0, 3.9, 12.9, 0.0, 0.0, 9.9, 6.9, 0.0, 0.0, 0.0, 13.9, 7.9, 7.9, 13.9, 2.9, 0.0, 0.0, 0.0, 4.9, 13.9, 13.9, 5.9, 0.0, 0.0]]))]
PROPERTY["8400.1"]=[pca.transform(np.array([[0.1, 0.1, 0.1, 2.1, 13.1, 4.1, 0.1, 0.1, 0.1, 0.1, 0.1, 7.1, 13.1, 2.1, 0.1, 0.1, 0.1, 0.1, 1.1, 13.1, 8.1, 2.1, 1.1, 0.1, 0.1, 0.1, 9.1, 12.1, 2.1, 8.1, 4.1, 0.1, 0.1, 4.1, 14.1, 6.1, 6.1, 14.1, 4.1, 0.1, 0.1, 11.1, 15.1, 13.1, 14.1, 14.1, 2.1, 0.1, 0.1, 5.1, 8.1, 9.1, 15.1, 10.1, 0.1, 0.1, 0.1, 0.1, 0.1, 3.1, 13.1, 5.1, 0.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 0.0, 1.9, 12.9, 3.9, 0.0, 0.0, 0.0, 0.0, 0.0, 6.9, 12.9, 1.9, 0.0, 0.0, 0.0, 0.0, 0.9, 12.9, 7.9, 1.9, 0.9, 0.0, 0.0, 0.0, 8.9, 11.9, 1.9, 7.9, 3.9, 0.0, 0.0, 3.9, 13.9, 5.9, 5.9, 13.9, 3.9, 0.0, 0.0, 10.9, 14.9, 12.9, 13.9, 13.9, 1.9, 0.0, 0.0, 4.9, 7.9, 8.9, 14.9, 9.9, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9, 12.9, 4.9, 0.0, 0.0]]))]
PROPERTY["8500.1"]=[pca.transform(np.array([[0.1, 0.1, 1.1, 12.1, 7.1, 1.1, 0.1, 0.1, 0.1, 0.1, 9.1, 14.1, 4.1, 0.1, 0.1, 0.1, 0.1, 0.1, 14.1, 7.1, 0.1, 0.1, 0.1, 0.1, 0.1, 2.1, 14.1, 3.1, 0.1, 0.1, 0.1, 0.1, 0.1, 4.1, 14.1, 7.1, 8.1, 4.1, 0.1, 0.1, 0.1, 2.1, 15.1, 14.1, 12.1, 14.1, 6.1, 0.1, 0.1, 0.1, 11.1, 13.1, 7.1, 11.1, 11.1, 0.1, 0.1, 0.1, 1.1, 10.1, 14.1, 13.1, 6.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 0.9, 11.9, 6.9, 0.9, 0.0, 0.0, 0.0, 0.0, 8.9, 13.9, 3.9, 0.0, 0.0, 0.0, 0.0, 0.0, 13.9, 6.9, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9, 13.9, 2.9, 0.0, 0.0, 0.0, 0.0, 0.0, 3.9, 13.9, 6.9, 7.9, 3.9, 0.0, 0.0, 0.0, 1.9, 14.9, 13.9, 11.9, 13.9, 5.9, 0.0, 0.0, 0.0, 10.9, 12.9, 6.9, 10.9, 10.9, 0.0, 0.0, 0.0, 0.9, 9.9, 13.9, 12.9, 5.9, 0.0]]))]
PROPERTY["8600.1"]=[pca.transform(np.array([[0.1, 0.1, 0.1, 5.1, 14.1, 7.1, 0.1, 0.1, 0.1, 0.1, 1.1, 12.1, 16.0, 10.1, 0.1, 0.1, 0.1, 2.1, 9.1, 15.1, 15.1, 7.1, 0.1, 0.1, 0.1, 5.1, 13.1, 14.1, 15.1, 6.1, 0.1, 0.1, 0.1, 2.1, 6.1, 10.1, 15.1, 5.1, 0.1, 0.1, 0.1, 0.1, 0.1, 8.1, 15.1, 6.1, 0.1, 0.1, 0.1, 0.1, 0.1, 8.1, 15.1, 9.1, 0.1, 0.1, 0.1, 0.1, 0.1, 5.1, 14.1, 10.1, 1.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 0.0, 4.9, 13.9, 6.9, 0.0, 0.0, 0.0, 0.0, 0.9, 11.9, 15.9, 9.9, 0.0, 0.0, 0.0, 1.9, 8.9, 14.9, 14.9, 6.9, 0.0, 0.0, 0.0, 4.9, 12.9, 13.9, 14.9, 5.9, 0.0, 0.0, 0.0, 1.9, 5.9, 9.9, 14.9, 4.9, 0.0, 0.0, 0.0, 0.0, 0.0, 7.9, 14.9, 5.9, 0.0, 0.0, 0.0, 0.0, 0.0, 7.9, 14.9, 8.9, 0.0, 0.0, 0.0, 0.0, 0.0, 4.9, 13.9, 9.9, 0.9, 0.0]]))]
PROPERTY["8700.1"]=[pca.transform(np.array([[0.1, 0.1, 4.1, 13.1, 4.1, 0.1, 0.1, 0.1, 0.1, 0.1, 12.1, 10.1, 1.1, 0.1, 0.1, 0.1, 0.1, 3.1, 14.1, 2.1, 0.1, 0.1, 0.1, 0.1, 0.1, 4.1, 14.1, 3.1, 4.1, 1.1, 0.1, 0.1, 0.1, 6.1, 14.1, 13.1, 13.1, 13.1, 2.1, 0.1, 0.1, 3.1, 15.1, 8.1, 2.1, 7.1, 11.1, 0.1, 0.1, 0.1, 13.1, 9.1, 2.1, 8.1, 12.1, 0.1, 0.1, 0.1, 4.1, 12.1, 15.1, 13.1, 5.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 3.9, 12.9, 3.9, 0.0, 0.0, 0.0, 0.0, 0.0, 11.9, 9.9, 0.9, 0.0, 0.0, 0.0, 0.0, 2.9, 13.9, 1.9, 0.0, 0.0, 0.0, 0.0, 0.0, 3.9, 13.9, 2.9, 3.9, 0.9, 0.0, 0.0, 0.0, 5.9, 13.9, 12.9, 12.9, 12.9, 1.9, 0.0, 0.0, 2.9, 14.9, 7.9, 1.9, 6.9, 10.9, 0.0, 0.0, 0.0, 12.9, 8.9, 1.9, 7.9, 11.9, 0.0, 0.0, 0.0, 3.9, 11.9, 14.9, 12.9, 4.9, 0.0]]))]
PROPERTY["8800.1"]=[pca.transform(np.array([[0.1, 0.1, 0.1, 7.1, 12.1, 0.1, 0.1, 0.1, 0.1, 0.1, 3.1, 15.1, 6.1, 0.1, 1.1, 0.1, 0.1, 0.1, 11.1, 11.1, 0.1, 3.1, 5.1, 0.1, 0.1, 4.1, 15.1, 3.1, 1.1, 11.1, 8.1, 0.1, 0.1, 9.1, 14.1, 6.1, 10.1, 15.1, 4.1, 0.1, 0.1, 8.1, 13.1, 14.1, 15.1, 11.1, 0.1, 0.1, 0.1, 1.1, 3.1, 7.1, 15.1, 2.1, 0.1, 0.1, 0.1, 0.1, 0.1, 8.1, 11.1, 0.1, 0.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 0.0, 6.9, 11.9, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9, 14.9, 5.9, 0.0, 0.9, 0.0, 0.0, 0.0, 10.9, 10.9, 0.0, 2.9, 4.9, 0.0, 0.0, 3.9, 14.9, 2.9, 0.9, 10.9, 7.9, 0.0, 0.0, 8.9, 13.9, 5.9, 9.9, 14.9, 3.9, 0.0, 0.0, 7.9, 12.9, 13.9, 14.9, 10.9, 0.0, 0.0, 0.0, 0.9, 2.9, 6.9, 14.9, 1.9, 0.0, 0.0, 0.0, 0.0, 0.0, 7.9, 10.9, 0.0, 0.0, 0.0]]))]
PROPERTY["8900.1"]=[pca.transform(np.array([[0.1, 0.1, 4.1, 13.1, 14.1, 14.1, 11.1, 3.1, 0.1, 0.1, 6.1, 9.1, 8.1, 11.1, 14.1, 3.1, 0.1, 0.1, 0.1, 0.1, 0.1, 11.1, 11.1, 0.1, 0.1, 0.1, 0.1, 3.1, 7.1, 15.1, 5.1, 0.1, 0.1, 0.1, 6.1, 12.1, 15.1, 13.1, 4.1, 0.1, 0.1, 1.1, 3.1, 9.1, 14.1, 4.1, 0.1, 0.1, 0.1, 0.1, 1.1, 14.1, 7.1, 0.1, 0.1, 0.1, 0.1, 0.1, 6.1, 13.1, 1.1, 0.1, 0.1, 0.1]])), pca.transform(np.array([[0.0, 0.0, 3.9, 12.9, 13.9, 13.9, 10.9, 2.9, 0.0, 0.0, 5.9, 8.9, 7.9, 10.9, 13.9, 2.9, 0.0, 0.0, 0.0, 0.0, 0.0, 10.9, 10.9, 0.0, 0.0, 0.0, 0.0, 2.9, 6.9, 14.9, 4.9, 0.0, 0.0, 0.0, 5.9, 11.9, 14.9, 12.9, 3.9, 0.0, 0.0, 0.9, 2.9, 8.9, 13.9, 3.9, 0.0, 0.0, 0.0, 0.0, 0.9, 13.9, 6.9, 0.0, 0.0, 0.0, 0.0, 0.0, 5.9, 12.9, 0.9, 0.0, 0.0, 0.0]]))]

In [54]:
for k in PROPERTY.keys():
    low = []
    high = []
    for i in range(len(PROPERTY[k][0][0])):
        low.append(min(PROPERTY[k][0][0][i],PROPERTY[k][1][0][i]))
        high.append(max(PROPERTY[k][0][0][i],PROPERTY[k][1][0][i]))
    print(f"PROPERTY[\"{k}\"]=[{str(high)},{str(low)}]\n")

PROPERTY["8000.1"]=[[69.93163175120667, -1.820589825127079, -20.86258628168296, 7.045107707924281, 13.961806364859358, 45.9233117393756, 27.204866984593476, 13.202061237154995, 37.194307742156404, 9.148477585163121, 14.555794574051081, -7.633955478973268, 17.24877574318151, 15.605699824483466, -0.4943379359474224, 4.670024215462356, -31.75146537387204, 3.397307425308147, 53.50271197384673, 33.7215630824975, 3.292792375369547, -1.1483049043454232, -4.558711473595673, 17.371587094278727, -30.01419971494068, -7.17935920099066, 103.45039922368152, 102.09192217280054, -63.16312677897202, 58.95013718040333, 251.4622885104744, -31.270133249306053, -105.8051838374209, -110.84843998123533, -64.46535905881149, -134.99753661929017, 100.63211504580569, 88.36802752011815, 291.44575891957106, 382.23215660985346, -178.44814385493316, 204.15782760057957, 39.66769014431959, -5.904093846288786, 207.0566901443647, 29.36701498433553, -5.654909500750741, 7.783758062419179, -38.997717146895674, 1.6455166331